# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

> Note: The 'llama-3.1-70b-versatil' model was deprecated. As a result, we updated it to the 'llama-3.3-70b-versatil' model in January 2025.

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.3-70b-versatile"

## Creating Crew, Agents, and Tasks

In [6]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

2025-11-05 04:43:30,304 - 140067414494080 - sqlite.py-sqlite:111 - WARNING: ⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


## Kicking off the Crew

In [7]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: To create a detailed summary report on the most impactful financial news and updates related to Inflation in the US and its impact on the stock market in 2024, I should start by gathering the latest information from reliable financial news sources and real-time market data tools. This will involve searching the internet for relevant news articles, reports



# Agent: Lead Market Analyst
## Final Answer: 
<title>Financial perspectives</title>
Is a Market Correction Coming? 
October 15, 2025

Investment outlook webinar
Year-end review: Tax law changes, investment outlook and your financial plan
Register now
6-min read
Share:
 Copy link to clipboard
Key takeaways
U.S. equity markets rebounded strongly in 2025, driven by solid company fundamentals and resilient consumer spending.
Tariff policies and government shutdowns create uncertainty, but investors are focused on stable economic data and earnings growth.
Diversification and long-term investment strategies remain crucial amid market volatility and shifting Federal Reserve interest rate policies.
U.S. equity markets softened from record highs after rebounding decisively from an early 2025 setback. 1 Since early April, when the S&P 500 narrowly avoided a bear market (a 20% decline or worse), investors have shrugged off higher tariffs and the ongoing government shutdown. Instead,  market pa



# Agent: Chief Data Strategist
## Thought: To analyze market data and trends related to inflation in the US and its impact on the stock market in 2024, I need to gather information from reliable sources. My first step would be to search the internet for relevant data and analysis.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation rate 2024 and stock market impact\"}"
## Tool Output: 

Search results: Title: United States Inflation Rate - Trading Economics
Link: https://tradingeconomics.com/united-states/inflation-cpi
Snippet: The annual inflation rate in the US rose to 3% in September 2025, the highest since January, from 2.9% in August and below forecasts of 3.1%.
---
Title: How Does Inflation Affect the Stock Market and Share Prices? - IG
Link: https://www.ig.com/en/trading-strategies/how-does-inflation-affect-the-stock-market-210423
Snippet: For traders taking a short-term view, there is evidence to suggest that higher inflation also tends to le

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.87it/s]




# Agent: Chief Data Strategist
## Thought: Title: What can history tell us about inflation and the stock market - UBS
Link: https://www.ubs.com/global/en/wealth-management/insights/chief-investment-office/inflation-and-the-stock-market.html
Snippet: ... inflation was around 5%, but since 1980 it has generally ranged between 2% and 4%.
Thought: Next, I should search in a specific website to gather more data about the US inflation rate in 2024 and its impact on the stock market.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"US inflation rate 2024 stock market\", \"website\": \"https://tradingeconomics.com/\"}"
## Tool Output: 
Relevant Content:
outlook. In this scenario, continued geopolitical conflicts and trade actions cause inflation to stay higher for longer, driving the Federal Reserve to hike rates.1 Scenarios Baseline (70%): Real GDP growth slowed overall in the first quarter of 2024, coming in very close to our previous forecast’s expectation

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.54it/s]




# Agent: Chief Data Strategist
## Thought: Therapy ETF Opportunity for Long-Term Wealth Creation August 29, 2024 Blockchain Revolution in Healthcare: A $12 Billion Opportunity for Disruptive Investors August 29, 2024 The Rise and Rise of 3D Printing Stocks August 28, 2024 August 28, 2024 Looming Rate Cuts Could Push These ETFs Higher August 28, 2024 Why Investors Should Buy These Top-Rated Dividend Aristocrat Stocks in a Bear Market August 28, 2024 Moving Averages: S&P Finishes August 2024 Up 2.3% August 28, 2024 Berkshire Hathaway vs Nvidia: The Battle Between Value & Growth August 28, 2024 Broadening Out Beyond ‘Mag 7’: Equal-Weight ETFs on a Roll By Kirsten Chang August 28, 2024 Two Measures of Inflation: July 2024 Jennifer NashAugust 28, 2024 The BEA's core Personal Consumption Expenditures (PCE) Price Index for July showed that core inflation continues to be above the Federal Reserve's 2% long-term target at 2.6%. The July core Consumer Price Index (CPI) release was higher, at 3



# Agent: Chief Data Strategist
## Final Answer: 
**Comprehensive Analysis Report:**

**Introduction:**
The US inflation rate in 2024 was around 3%, which is above the Federal Reserve's 2% long-term target. This had an impact on the stock market, with the S&P 500 trending higher between November 2023 and November 2024, following a general downward trend between August and October 2023.

**Key Findings:**

1. **Inflation and Stock Market:** The US inflation rate in 2024 was around 3%, which is above the Federal Reserve's 2% long-term target. This had an impact on the stock market, with the S&P 500 trending higher between November 2023 and November 2024.
2. **Diversification:** Determining the right asset mix for a portfolio involves making investment decisions that suit an individual's personal situation, not just reacting to market and economic trends.
3. **Long-term Investment Strategy:** Investors should take a long-range view and avoid the temptation to time the market.

**Data Vis



# Agent: Creative Content Director
## Thought: Thought: I need to perform a search focused on the specific topic of US inflation and its impact on the stock market for the year 2024 using the correct format.
## Using tool: Search the internet
## Tool Input: 
"{\"name\": \"Search the impact of US inflation on stock market 2024\", \"description\": \"\", \"args_schema\": {}, \"return_direct\": false, \"verbose\": false}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Creative Content Director
## Thought: Thought: It appears that the formatting for input has not been successfully followed. I will attempt to reset my approach to perform the search effectively.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation trends and stock market impact in 2024\"}"
## Tool Output: 

Search results: Title: United States Economic Forecast Q3 2025 - Deloitte
Link: https://www.deloitte.com/us/en/insights/topics/economy/us-econom

## Social Content

In [8]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
Did you realize that a 3% inflation rate can lead
to increased stock market volatility? It's crucial
now more than ever to stay informed and diversify
your portfolio effectively! #InvestSmart
#StockMarket #Inflation
--------------------------------------------------


## Blog Post

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

### **Understanding Inflation in the US and Its Impact on the Stock Market in 2024**

#### **Blog Post: The Inflation Landscape: Navigating the Stock Market in 2024**

**Introduction**  
In 2024, the US experienced inflation at approximately 3%, marking a significant shift from previous years and rising above the Federal Reserve's long-term target of 2%. This inflation spike has implications for investors, affecting everything from purchasing power to stock market performance. Investors need to stay informed about inflation trends to optimize their portfolios and mitigate risks associated with market volatility.

Inflation impacts not only everyday consumer prices but also the valuation of assets in the stock market. As inflation persists, costs of goods and services rise, prompting consumers to adjust their spending habits, which in turn affects corporate earnings. Thus, understanding this complex relationship becomes crucial for making informed investment decisions in an evolving economic landscape.

**Key Findings:**  
1. **Inflation and Its Correlation with Stock Markets:**  
   With inflation rates hovering around 3%, the S&P 500 exhibited resilience, although increased volatility was noted—a typical reaction to inflationary pressures. Historically, periods of heightened inflation can lead to choppy market conditions, compelling investors to reassess their strategies to navigate potential downturns.

2. **Diversification as a Strategy:**  
   Investors are reminded of the importance of a diversified portfolio. Ensuring a balanced approach across various asset classes, including equities, fixed income, and alternative investments, is essential for weathering inflation-induced market fluctuations. A diversified portfolio can provide a cushion against volatility and reduce overall risk, allowing investors to capitalize on potential growth opportunities.

3. **Long-Term Investment Mindset:**  
   Amidst inflation fears, experts advocate for a long-range investment strategy, emphasizing that timing the market can lead to missed opportunities. By maintaining a focus on long-term goals, investors can ride out short-term volatility, benefiting from compounding returns over time. This approach encourages buying and holding quality stocks rather than reacting impulsively to market fluctuations.

#### **Data Visualization:**
- **Infographic Idea:** **"The Impact of Inflation on Stock Market Trends"**  
  This infographic could illustrate the relationship between inflation rates and significant stock market fluctuations over the past decade. Highlighting key events where inflation influenced the market will provide context, enabling traders to understand how to navigate these changes effectively.

#### **Social Media Update:**
- **Post Example:**  
  "Did you realize that a 3% inflation rate can lead to increased stock market volatility? It's crucial now more than ever to stay informed and diversify your portfolio effectively! #InvestSmart #StockMarket #Inflation"

#### **Video Content:**
- **Proposed Video:** **"Long-Term Investment Strategies in an Inflationary Environment"**  
  A short video discussing strategies like dollar-cost averaging, risk assessment, and how to adjust asset allocations based on shifting economic indicators will serve as an invaluable resource for investors aiming to stay ahead of the curve.

#### **Conclusion:**  
As we look forward to 2024, understanding the interplay between inflation and the stock market is vital for all investors. The past year has taught us that being proactive with our investment strategies, confirming asset allocations suited to our financial goals, and preparing for market volatility provides an edge. For tailored advice, reaching out to a financial planner is recommended to develop a personalized investment strategy that can weather economic uncertainties.

---

This detailed blog post, along with integrated multimedia ideas such as infographics and video content, will engage and educate readers about the critical relationship between US inflation and the stock market in 2024. The content clearly conveys the key findings and recommendations relevant to current trends in inflation and its implications for investing, promoting informed decision-making in a dynamic financial environment.